In [1]:
import math as m
import numpy as np
import pytest

In [2]:
def From_Kep_to_Dec(kep, mu):
    dec = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'd')
    #array<double,6>dec;//первый вектор отвечает за координаты, а второй за скорость
    B = np.array(range(9), 'd').reshape((3,3))#матрица перехода от орбитальных координат к декартовым
    
    if kep[3] >= 1:
        raise ValueError('The orbit is not elliptical')
    kep[0] = m.radians(kep[0])
    kep[1] = m.radians(kep[1])
    kep[4] = m.radians(kep[4])
    kep[5] = m.radians(kep[5])

    E = 2*m.atan(m.sqrt((1-kep[3])/(1+kep[3]))*m.tan(kep[5]/2))#эксцентричная аномалия


    B[0][0] = m.cos(kep[1]) * m.cos(kep[4]) - m.sin(kep[1]) * m.sin(kep[4]) * m.cos(kep[0])
    B[0][1] = -m.cos(kep[1]) * m.sin(kep[4]) - m.sin(kep[1]) * m.cos(kep[4]) * m.cos(kep[0])
    B[0][2] = m.sin(kep[1]) * m.sin(kep[0])

    B[1][0] = m.sin(kep[1]) * m.cos(kep[4]) + m.cos(kep[1]) * m.sin(kep[4]) * m.cos(kep[0])
    B[1][1] = -m.sin(kep[1]) * m.sin(kep[4]) + m.cos(kep[1]) * m.cos(kep[4]) * m.cos(kep[0])
    B[1][2] = -m.cos(kep[1]) * m.sin(kep[0])


    B[2][0] = m.sin(kep[4]) * m.sin(kep[0])
    B[2][1] = m.cos(kep[4]) * m.sin(kep[0])
    B[2][2] = m.cos(kep[0])
    
    kep[0] = m.degrees(kep[0])
    kep[1] = m.degrees(kep[1])
    kep[4] = m.degrees(kep[4])
    kep[5] = m.degrees(kep[5])


    b_1 = B[:,0]
    b_2 = B[:,1]
    


    r = (b_1*(m.cos(E) - kep[3]) + b_2*m.sqrt(1.0 - kep[3] * kep[3])*m.sin(E))*kep[2]
    dec[:3] = r


    tau = np.array([-m.sin(E)/m.sqrt(1.0 - kep[3]*kep[3]), m.cos(E), 0.0], 'd')
    tau = tau/m.sqrt(tau[0]*tau[0] + tau[1]*tau[1])


    tau = B[:,0]*tau[0] + B[:,1]*tau[1] + B[:,2]*tau[2]

    abs_V = m.sqrt(mu * (2.0/m.sqrt(r[0] * r[0] + r[1] * r[1] + r[2] * r[2]) - 1.0/kep[2]))
    V = tau * abs_V

    dec[3:6] = V[:]

    return dec

In [3]:
def From_Dec_to_Kep(r, v, mu):
    r = np.array(r,'d')
    v = np.array(v,'d')
    kep = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])#(i, Omega, a, e, om, nu) =
                        #(наклонение, долгота восходящего угла, большая полуось,
                        #эксцентриситет, аргумент перецентра, истинная аномалия)
    r_vect_mult_v = np.cross(r,v)
    abs_r_vect_mult_v = m.sqrt(np.dot(r_vect_mult_v,r_vect_mult_v))
    k = r_vect_mult_v/abs_r_vect_mult_v
    kep[0] = m.acos(k[2])#наклонение

    if kep[0] == 0.0 :
        kep[1] = -10000.0# долгота восходящего узла не определена
    else:
        kep[1] = m.acos(-k[1]/m.sin(kep[0])) # долгота восходящего узла
        if k[0]/m.sin(kep[0]) < 0 :
            kep[1] = 2 * m.pi - kep[1]
            
    abs_v = m.sqrt(np.dot(v,v))
    abs_r = m.sqrt(np.dot(r,r))
    h = abs_v*abs_v - 2 * mu/abs_r #константа энергии

    c = np.cross(r,v) #вектор константы скорости
    p = np.dot(c,c)/mu #фокальный параметр
    if h >= 0:
        raise ValueError('The orbit is not elliptical')
        
    kep[3] = m.sqrt(1.0 + h*np.dot(c,c)/(mu*mu)) #эксцентриситет
    kep[2] = p / (1.0 - kep[3] * kep[3])

    #Найдём истинную аномалию
    kep[5] = m.acos((1.0/kep[3]) * (p/abs_r - 1.0))

    if np.dot(r,v) < 0.0:
        kep[5] = 2*m.pi - kep[5]
    elif np.dot(r,v) == 0:
        raise ValueError('Undefined anomaly')

    if k[2] != 1 :
        #Найдем аргумент перецентра
        normir_r = r/abs_r

        I = np.array([1.0, 0.0, 0.0],'d')
        J = np.array([0.0, 1.0, 0.0],'d')

        normir_b = I * m.cos(kep[1]) + J * m.sin(kep[1])
        cos_sum = np.dot(normir_r, normir_b)
        sin_sum = np.dot(k, np.cross(normir_b, normir_r))

        if sin_sum >= 0:
            omega = m.acos(cos_sum) - kep[5]
        else:
            omega = 2*m.pi - m.acos(cos_sum) - kep[5]

        if omega < 0:
            omega += 2*m.pi
        elif 2*m.pi < omega:
            omega -= 2*m.pi

        kep[4] = omega
    else:
        kep[4] = -10000.0

    kep[0] = m.degrees(kep[0])
    kep[1] = m.degrees(kep[1])
    kep[4] = m.degrees(kep[4])
    kep[5] = m.degrees(kep[5])


    return kep

In [4]:
From_Dec_to_Kep([1.306e8, -1.979e8, 1.837e8], [24.004, -6.837, -10.532], 132712440018.0)
#[60.0, 150.0, 8.794e8, 0.7, 90.0, 45.0]

array([5.99953155e+01, 1.49998066e+02, 8.78570930e+08, 6.99760326e-01,
       8.99933271e+01, 4.49969470e+01])

In [5]:
def rel_error(t, u):
    er = 0.0
    for i in range(len(t)):
        if u[i] * t[i] != 0.0:
            er = max(er, abs((t[i]-u[i])/t[i]))
        else:
            er = max(er,max(abs(t[i]), abs(u[i])))
    return er


In [6]:
def AssertEqual (t, u):
    epsilon = rel_error(t, u)
    if epsilon > 0.001: 
        return False
    return True

In [7]:
def test_for_kep_to_dec():
    mu_s = 132712440018
    #Тест 1
    a = From_Kep_to_Dec([60, 150, 8.794e8, 0.7, 90, 45], mu_s)
    b = [1.306e8, -1.979e8, 1.837e8,24.004, -6.837, -10.532]
    assert AssertEqual(a,b)
    #Тест 2
    a = From_Kep_to_Dec([120, 60, 8.794e8, 0.7, 60, 135], mu_s)
    b = [-5.284e8, -6.854e8, -1.991e8, -7.576, -2.527, -9.176]
    assert AssertEqual(a,b)
    #Тест 3
    a = From_Kep_to_Dec([60.0, 240.0, 8.794e8, 0.7, 150.0, 225.0], mu_s)
    b = [-3.294e8, -8.003e8, 1.991e8, 7.916, 7.523, 5.359]
    assert AssertEqual(a,b)
    #Тест 4
    a = From_Kep_to_Dec([60.0, 300.0, 8.794e8, 0.7, 225.0, 300.0], mu_s)
    b = [-1.232e8, 2.994e8, 7.446e7, -8.851, -9.801, -21.764]
    assert AssertEqual(a,b)
    #Тест 5
    a = From_Kep_to_Dec([120.0, 180.0, 8.794e8, 0.7, 300.0, 270.0], mu_s)
    b = [3.884e8, -1.121e8, -1.942e8, -19.029, -4.438, -7.687]
    assert AssertEqual(a,b)

In [8]:
def test_for_dec_to_kep():
    mu_s = 132712440018
    #Тест 1
    r = [1.306e8, -1.979e8, 1.837e8]
    v = [24.004, -6.837, -10.532]
    a = From_Dec_to_Kep(r,v, mu_s)
    b = [60.0, 150.0, 8.794e8, 0.7, 90.0, 45.0]
    assert AssertEqual(a,b)
    #Тест 2
    r = [-5.284e8, -6.854e8, -1.991e8]
    v = [-7.576, -2.527,-9.176]
    a = From_Dec_to_Kep(r,v, mu_s)
    b = [120.0, 60.0, 8.794e8, 0.7, 60.0, 135.0]
    assert AssertEqual(a,b)
    #Тест 3
    r = [-3.294e8, -8.003e8, 1.991e8]
    v = [7.916, 7.523, 5.359]
    a = From_Dec_to_Kep(r,v, mu_s)
    b = [60.0, 240.0, 8.794e8, 0.7, 150.0, 225.0]
    assert AssertEqual(a,b)
    #Тест 4
    r = [-1.232e8, 2.994e8, 7.446e7]
    v = [-8.851, -9.801, -21.764]
    a = From_Dec_to_Kep(r,v, mu_s)
    b = [60.0, 300.0, 8.794e8, 0.7, 225.0, 300.0]
    assert AssertEqual(a,b)
    #Тест 5
    r = [3.884e8, -1.121e8, -1.942e8]
    v = [-19.029, -4.438, -7.687]
    a = From_Dec_to_Kep(r,v, mu_s)
    b = [120.0, 180.0, 8.794e8, 0.7, 300.0, 270.0]
    assert AssertEqual(a,b)

In [9]:
test_for_kep_to_dec()
test_for_dec_to_kep()